In [ ]:
import cobra
from cobra.io import read_sbml_model, write_sbml_model
from cobra.flux_analysis import fastcc
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import  find_blocked_reactions
import itertools as it
import csv
import os
import re

In [ ]:
# load draft reconstructions
reconstruction_imat = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_iMAT_fastcc.xml')
reconstruction_tinit = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_tINIT_fastcc.xml')
reconstruction_CORDA = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_CORDA_fastcc.xml')
reconstruction_pymcadre = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_pymcadre_fastcc.xml')

In [ ]:
# Extract info of reactions
rxn_i = [reconstruction_imat.reactions[i].id for i in range(len(reconstruction_imat.reactions))]
rxn_t = [reconstruction_tinit.reactions[i].id for i in range(len(reconstruction_tinit.reactions))]
rxn_c = [reconstruction_CORDA.reactions[i].id for i in range(len(reconstruction_CORDA.reactions))]
rxn_p = [reconstruction_pymcadre.reactions[i].id for i in range(len(reconstruction_pymcadre.reactions))]

In [ ]:
# obtain all reactions in the four draft reconstruction
all_rxn = set(rxn_t) | set(rxn_i) | set(rxn_c) | set(rxn_p)


In [ ]:
# Obtain reactions unique in tINIT draft reconstruction
tINIT_U = set(rxn_t) - set(rxn_i)-set(rxn_c)-set(rxn_p)
len(tINIT_U)

In [ ]:
# Reomve reactions unique to tINIT from all reactions
consensus_rxn = all_rxn - tINIT_U
len(consensus_rxn)

In [ ]:
General_model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')
for i in range (len(General_model.reactions)):
    if General_model.reactions[i].id.startswith('EX_'):
        General_model.reactions[i].bounds = (-100, 100)

In [ ]:
consensus_reconstruction  = cobra.Model('Consensus Reconstruction')

In [ ]:
for i in consensus_rxn:
    consensus_reconstruction.add_reactions([General_model.reactions.get_by_id(i)])

In [ ]:
write_sbml_model(consensus_reconstruction, '/home/jiang817/cybergut_project/Consistent_Reconstruction/consensus_reconstruction.xml')

In [ ]:
# fastcc test
consensus_reconstruction_fastcc = fastcc(consensus_reconstruction)

In [ ]:
write_sbml_model(consensus_reconstruction_fastcc, '/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/consensus_reconstruction_fastcc.xml')